In [1]:
import psycopg2
from zipfile import ZipFile
import os

In [2]:
#1 creer un DB avec postgres

#2 creer une table des lettres T_lettres



In [3]:
def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    print(chemin)
    return chemin

chemin_f=chemin_fichier()

/home/tatindom/Santa-Data/santa-Groupe-DZJC/Dossier_de_travail


In [5]:
# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn
                

In [6]:
def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [7]:
def create_table(conn, create_table_psg : str,table_name):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_psg)
        print(f"Table created successfully in PostgreSQL : {table_name}")
    except (Exception, psycopg2.DatabaseError) as error :
        print(f"Error while creating PostgreSQL table : {table_name}", error)
    finally:
        #fermer le curseur
        if(conn):
            c.close()

In [8]:
# MAIN
# CONNECTION A LA BASE

conn=connexion_DB("santa_data")
#print(conn)

Proprietes :  {'user': 'santa', 'dbname': 'santa_data', 'host': 'localhost', 'port': '5432:5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Version de PostgreSQL :  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 



In [9]:
#creation de tables
if conn is not None:
    #T_letters
    nm_table='T_letters'
    
    c = conn.cursor()
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_l SERIAL PRIMARY KEY NOT NULL,
            Name_letter TEXT NOT NULL
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()

    #T_enfants
    nm_table='T_enfants'
    
    c = conn.cursor()
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_e SERIAL PRIMARY KEY,
            Name_enfant TEXT NOT NULL,
            Age_enfant FLOAT,
            Pays_enfant TEXT,
            ID_p INT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()
    
    #T_contenu_lettres
    nm_table='T_contenus'
    
    c = conn.cursor()
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_c SERIAL PRIMARY KEY,
            ID_l INT,
            ID_enfant INT,
            Contenu TEXT,
            Numero INT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()
    
    #T_pays
    nm_table='T_pays'
    
    c = conn.cursor()
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID_p SERIAL PRIMARY KEY,
            Pays TEXT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()
    
    #T_complete
    nm_table='T_complete'
    
    c = conn.cursor()
    c.execute(f"DROP TABLE IF EXISTS {nm_table}")
    
    create_table_query = f'''
            CREATE TABLE IF NOT EXISTS {nm_table}
            (ID SERIAL PRIMARY KEY,
            id_l INT,
            nom_lettre TEXT,
            nom_enfant TEXT,
            age TEXT,
            Pays TEXT,
            Contenu TEXT,
            Numero INT
            ); 
            '''
    create_table(conn,create_table_query,nm_table)
    conn.commit()


Table created successfully in PostgreSQL : T_letters
Table created successfully in PostgreSQL : T_enfants
Table created successfully in PostgreSQL : T_contenus
Table created successfully in PostgreSQL : T_pays
Table created successfully in PostgreSQL : T_complete


In [10]:
deconnexion_DB(conn)

PostgreSQL connection is closed


In [11]:
#insertion de données
conn=connexion_DB("santa_data")
try:
    conn=connexion_DB("santa_data")
    cursor = conn.cursor()

    path_dossier_letters=chemin_fichier() +'/lettres_pere_noel'
    #print(path_dossier_letters)
#'/home/tatindom/Santa-Data/santa-Groupe-DZJC/Partie_1/lettres_pere_noel')
    
    #recuperation du nom des fichiers <lettre---.pnoel>
    list_letters=os.listdir(path_dossier_letters)
    list_letters.sort()
       
    count=0
    
    for f_name in list_letters :
        #print(f_name)
        insert_data_query = f""" 
                        INSERT INTO T_letters
                        (ID_l,Name_letter) 
                        VALUES (default,'{f_name}')
                        """
        cursor.execute(insert_data_query)
        #nb_l+=1
        conn.commit()
        count += cursor.rowcount
    print (count, "Record inserted successfully into mobile table")
    
    cursor.execute("SELECT * FROM T_letters")
    #resultats=cur.execute(select_SQL)
    res=cursor.fetchall()
    print(res)
    
except (Exception, psycopg2.Error) as error :
    if(conn):
        print("Failed to insert record into table T_letters", error)

finally:
    #closing database connection.
    if(conn):
        cursor.close()
        deconnexion_DB(conn)

Proprietes :  {'user': 'santa', 'dbname': 'santa_data', 'host': 'localhost', 'port': '5432:5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Version de PostgreSQL :  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

Proprietes :  {'user': 'santa', 'dbname': 'santa_data', 'host': 'localhost', 'port': '5432:5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

Version de PostgreSQL :  ('PostgreSQL 12.1 (Debian 12.1-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

/home/tatindom/Santa-Data/santa-Groupe-DZJC/Dossier_de_travail
1640 Record inserted successfully into mobile table
[(1, 'lettre_1.pnoel'), (2, 'lettre_10.pnoel'), (3, 'lettre_100.pnoel'), (4, 'lettre_1000.pnoel'), (5, 'lettre_1001.pnoel'), (6, 'lettre_1002

In [ ]:
version: "3"

services:
  postgres:
    image: "postgres:latest"
    environment:
      POSTGRES_USER: "santa"
      POSTGRES_PASSWORD: "claus"
      POSTGRES_DB: "santa_data"
    volumes:
      - "./data/postgres:/var/lib/postgresql/data"
    ports:
      - "5432:5432"

In [ ]:
     
      
    # spécifiant le nom du fichier zip
    file = "lettres_pere_noel.zip"
      
    # ouvrir le fichier zip en mode lecture
    with ZipFile(file, 'r') as zip: 
        # afficher tout le contenu du fichier zip
        zip.printdir() 
      
        # extraire tous les fichiers
        print('extraction...') 
        zip.extractall() 
        print('Terminé!')
        

In [ ]:
#recuperer la liste des fichiers d'un dossier
list_lettres=os.listdir('/home/tatindom/Santa-Data/santa-Groupe-DZJC/Partie_1/lettres_pere_noel')
print(list_lettres)

In [ ]:
print(list_lettres)